# Problem 1(a).  Reading Amazon Reviews.

In this problem, we will analyze Amazon reviews to determine what characteristics make them most helpful.

Download the file of Amazon gourmet food reviews from the [Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/web-FineFoods.html).   (Your computer may already have a utility installed that can unzip the archive as a text file; if not, [7-zip](http://www.7-zip.org/) is a useful utility for Windows. You can also do a web search for ```open .gz files online``` and use an online utility.)

Create a pandas DataFrame object with the following entries for each review:

* Product ID
* Number of people who voted this review helpful
* Total number of people who rated this review
* Rating of the product
* Text of the review

For the second and third of these, the information will be given in the dataset as ```1/5```, which would correspond to 1 vote for helpful out of 5 people who rated the review.


In [73]:
import pandas
import numpy
import re
import io

In [74]:
with open("foods.txt") as ff:
    food = ff.read()
    food = food.replace('\n','.')
    food = food.replace(',','.')
    food = food.replace('product/productId: ','\n')
    food = food.replace('review/userId: ',',')
    food = food.replace('review/profileName: ',',')
    food = food.replace('review/helpfulness: ',',')
    food = food.replace('review/score: ',',')
    food = food.replace('review/time: ',',')
    food = food.replace('review/summary: ',',')
    food = food.replace('review/text: ',',')
    food = food[1:]
    foodheader = 'prodID,userID,profname,helpfullness,score,time,summary,text,1,2,3,4,5,6,7,8\n'
    food = foodheader+food
    food = io.StringIO(food)
    fooddf = pandas.read_csv(food)

C:\Anaconda2\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [75]:
fooddf= fooddf[['prodID','helpfullness','score','text']]
fooddf.head()

,prodID,helpfullness,score,text
0,B001E4KFG0.,1/1.,5.0.,I have bought several of the Vitality canned d...
1,B00813GRG4.,0/0.,1.0.,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0.,1/1.,4.0.,This is a confection that has been around a fe...
3,B000UA0QIQ.,3/3.,2.0.,If you are looking for the secret ingredient i...
4,B006K2ZZ7K.,0/0.,5.0.,Great taffy at a great price. There was a wid...


In [76]:
fooddf['helpfull'] = fooddf['helpfullness'].apply(lambda x: x.split('/')[0])
fooddf['numPeop'] = fooddf['helpfullness'].apply(lambda x: x.split('/')[1])
fooddf= fooddf[['prodID','score','text','helpfull','numPeop']]
fooddf['helpfull']=pandas.to_numeric(fooddf['helpfull'])
fooddf['numPeop']=pandas.to_numeric(fooddf['numPeop'])
fooddf.head()

,prodID,score,text,helpfull,numPeop
0,B001E4KFG0.,5.0.,I have bought several of the Vitality canned d...,1,1.0
1,B00813GRG4.,1.0.,Product arrived labeled as Jumbo Salted Peanut...,0,0.0
2,B000LQOCH0.,4.0.,This is a confection that has been around a fe...,1,1.0
3,B000UA0QIQ.,2.0.,If you are looking for the secret ingredient i...,3,3.0
4,B006K2ZZ7K.,5.0.,Great taffy at a great price. There was a wid...,0,0.0


In [77]:
fooddf['text']=fooddf['text'].apply(str)

# Problem 1(b).  Analyzing review text.

Add columns to your DataFrame for the length of a review, the number of exclamation points in a review, and the fraction of people who rated a review helpful. You should calculate the fraction who rated a review helpful using the two columns you made in 1a, and a ratio of 1 helpful rating out of 5 total ratings should be entered as 0.2, not the string ```1/5```. If no people voted on whether a problem was helpful, the corresponding entry in your percentage helpful column should be ```NaN```.

In [78]:
def countex(x):
    count=0
    for j in x:
        if j=='!':
            count+=1
    return count

In [79]:
def pctHelpfull(x):
    if x['numPeop']==0:
        return numpy.nan
    else:
        return x['helpfull']/float(x['numPeop'])
    

In [80]:
fooddf['lengReview'] = fooddf['text'].apply(lambda x: len(x))
fooddf['num!'] = fooddf['text'].apply(lambda x: countex(x))
fooddf['pctHelpfull'] = fooddf.apply( pctHelpfull,axis=1)
fooddf.head()

,prodID,score,text,helpfull,numPeop,lengReview,num!,pctHelpfull
0,B001E4KFG0.,5.0.,I have bought several of the Vitality canned d...,1,1.0,265,0,1.0
1,B00813GRG4.,1.0.,Product arrived labeled as Jumbo Salted Peanut...,0,0.0,192,0,NaN
2,B000LQOCH0.,4.0.,This is a confection that has been around a fe...,1,1.0,511,0,1.0
3,B000UA0QIQ.,2.0.,If you are looking for the secret ingredient i...,3,3.0,221,0,1.0
4,B006K2ZZ7K.,5.0.,Great taffy at a great price. There was a wid...,0,0.0,142,0,NaN


# Problem 1(c).  Summary statistics.

How many reviews are in the data set?  What is the average length of a review (in characters)?  What is the average rating?  What is the greatest number of exclamation marks used in a single review?  Use the pandas package to answer these questions, then summarize your results in a markdown cell.

In [81]:
#length of dataset
len(fooddf)

568431

In [82]:
#Avg. length of review
fooddf['lengReview'].mean()

438.22219055610969

In [83]:
#Avg. Rating
fooddf['score'] = fooddf['score'].apply(lambda x: x[:-1])
fooddf['score'] = pandas.to_numeric(fooddf['score'])
fooddf['score'].mean()

4.1831919793255468

In [84]:
#Max ! in review
fooddf['num!'].max()

84

Leng of dataset: ***568431*** 
Avg length of review: ***438.22*** 
Avg rating: ***4.18***
Max ! in one review: ***84***

# Problem 1(d).  Export.

Save your DataFrame as a .csv file suitable for future analysis in R.  Your .csv file should not include the review text column, as the presence of commas and quotation marks will make reading the file difficult.  You should also convert entries from ```NaN``` to the empty string before saving.

In [86]:
dfout = fooddf.replace(numpy.nan,' ', regex=True)
dfout = fooddf[['prodID','score','helpfull','numPeop','lengReview','num!','pctHelpfull']]
dfout.to_csv('Amazon_reviews.csv')